In [1]:
!echo '数据集行数:'
!wc -l 'bioCorpus_5000.txt'
!echo '======'
!echo '数据集前10行'
!head -10 'bioCorpus_5000.txt'

数据集行数:
    5000 bioCorpus_5000.txt
数据集前10行
formate assay in body fluids  application in methanol poisoning.
delineation of the intimate details of the backbone conformation of pyridinenucleotide coenzymes in aqueous solution.
metal substitutions incarbonic anhydrase  a halide ion probe study.
effect of chloroquine on cultured fibroblasts  release of lysosomal hydrolasesand inhibition of their uptake.
atomic models for the polypeptide backbones of myohemerythrin and hemerythrin.
studies of oxygen binding energy to hemoglobin molecule.
maturation of the adrenal medulla--IV
effects of morphine.
comparison between procaine and isocarboxazid metabolism in vitro by a livermicrosomal amidase-esterase.
radiochemical assay of glutathione S-epoxide transferase and its enhancement byphenobarbital in rat liver in vivo.


In [1]:
import gensim
from gensim.models import word2vec

##### Word2vec 训练

In [2]:
# 用生成器的方式读取文件里的句子
# 适合读取大容量文件，而不用加载到内存
class MySentences(object):
    def __init__(self, fname):
        self.fname = fname
 
    def __iter__(self):
        for line in open(self.fname,'r'):
            yield line.split()

In [3]:
# 模型训练函数
def w2vTrain(f_input, model_output):         
    sentences = MySentences(DataDir + f_input)
    w2v_model = word2vec.Word2Vec(sentences, 
                                  min_count = MIN_COUNT, 
                                  workers = CPU_NUM, 
                                  size = VEC_SIZE,
                                  window = CONTEXT_WINDOW)
    w2v_model.save(ModelDir + model_output)

In [5]:
# 训练
DataDir = "./"
ModelDir = "./ipynb_garbage_files/"
MIN_COUNT = 4
CPU_NUM = 2 # 需要预先安装 Cython 以支持并行
VEC_SIZE = 20
CONTEXT_WINDOW = 5 # 提取目标词上下文距离最长5个词
f_input = "bioCorpus_5000.txt"
model_output = "test_w2v_model"
w2vTrain(f_input, model_output)

In [6]:
w2v_model = word2vec.Word2Vec.load(ModelDir+model_output)

In [7]:
w2v_model.most_similar('body')

<ipython-input-7-4d5b9b7691d0>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_model.most_similar('body')


[('mouse', 0.9994620084762573),
 ('action', 0.9993744492530823),
 ('rat', 0.9993621110916138),
 ('studies', 0.9993410706520081),
 ('to', 0.9993181228637695),
 ('A', 0.9992624521255493),
 ('from', 0.9992518424987793),
 ('.', 0.9992084503173828),
 ('as', 0.9991968274116516),
 ('transport', 0.9991931915283203)]

In [10]:
# 停止词
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
StopWords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mxchip/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
StopWords[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [12]:
# 重新训练
# 模型训练函数
def w2vTrain_removeStopWords(f_input, model_output):
    sentences = list(MySentences(DataDir+f_input))
    for idx,sentence in enumerate(sentences):
        sentence = [w for w in sentence if w not in StopWords]
        sentences[idx]=sentence
        w2v_model = word2vec.Word2Vec(sentences, min_count = MIN_COUNT, 
                                      workers = CPU_NUM, size = VEC_SIZE)
        
    w2v_model.save(ModelDir+model_output)

In [ ]:
w2vTrain_removeStopWords(f_input, model_output)
w2v_model = word2vec.Word2Vec.load(ModelDir+model_output)

In [ ]:
w2v_model.most_similar('body')